In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [2]:
# Define paths
train_path = 'data/seg_train/seg_train'
test_path = 'data/seg_test/seg_test'
pred_path = 'data/seg_pred/seg_pred'

# List categories
categories = os.listdir(train_path)
print("Categories:", categories)

# Load images and labels
def load_images_and_labels(path, categories, img_size=(64, 64)):
    images = []
    labels = []
    for category in categories:
        category_path = os.path.join(path, category)
        for image_name in os.listdir(category_path):
            image_path = os.path.join(category_path, image_name)
            img = Image.open(image_path).resize(img_size)
            img = np.array(img) / 255.0  # Normalize to [0, 1]
            images.append(img)
            labels.append(category)
    return np.array(images), np.array(labels)

# Load training data
X_train, y_train = load_images_and_labels(train_path, categories)

# Convert labels to one-hot encoding
label_to_index = {label: idx for idx, label in enumerate(categories)}
y_train = np.array([label_to_index[label] for label in y_train])
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(categories))

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Load test data
X_test, y_test = load_images_and_labels(test_path, categories)
y_test = np.array([label_to_index[label] for label in y_test])
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(categories))

Categories: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [3]:
def custom_dropout(x, rate):
    """
    Custom dropout function.

    Args:
    - x: Input tensor.
    - rate: Dropout rate (fraction of units to drop).

    Returns:
    - Tensor with dropout applied.
    """
    # Generate a binary mask with the same shape as x
    mask = np.random.binomial(1, 1 - rate, size=x.shape)
    # Scale the mask to maintain the expected value of the input
    mask = mask / (1 - rate)
    # Apply the mask to the input
    return x * mask

def build_cnn_model(input_shape, num_classes, dropout_rate=0.5):
    model = models.Sequential([
        # Conv1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        # Conv2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        # Flatten
        layers.Flatten(),

        # FC1
        layers.Dense(128, activation='relu'),

        # Output
        layers.Dense(num_classes, activation='softmax')
    ])
    return model



# Initialize the model
input_shape = (64, 64, 3)  # Image size: 64x64, 3 color channels
num_classes = len(categories)
cnn_model = build_cnn_model(input_shape, num_classes)

# Compile the model
cnn_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Print model summary
cnn_model.summary()

c:\Users\Tsegaye\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,117,446 (8.08 MB)

 Trainable params: 2,117,446 (8.08 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Custom training loop with dropout
epochs = 10
batch_size = 32
dropout_rate = 0.5

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    # Shuffle the training data
    indices = np.arange(X_train.shape[0])
    np.random.shuffle(indices)
    X_train_shuffled = X_train[indices]
    y_train_shuffled = y_train[indices]

    # Train in batches
    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train_shuffled[i:i+batch_size]
        y_batch = y_train_shuffled[i:i+batch_size]

        with tf.GradientTape() as tape:
            # Apply custom dropout to the input
            X_batch_dropped = custom_dropout(X_batch, dropout_rate)
            # Forward pass
            logits = cnn_model(X_batch_dropped, training=True)
            # Compute the loss
            loss = tf.keras.losses.categorical_crossentropy(y_batch, logits)

        # Compute gradients
        gradients = tape.gradient(loss, cnn_model.trainable_variables)
        # Update weights
        cnn_model.optimizer.apply_gradients(zip(gradients, cnn_model.trainable_variables))

    # Evaluate on validation data
    val_loss, val_accuracy = cnn_model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Loss: {val_loss}")
    print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/10
Validation Loss: 1.2490686178207397
Validation Accuracy: 0.5600284934043884
Epoch 2/10
Validation Loss: 1.1667423248291016
Validation Accuracy: 0.5521909594535828
Epoch 3/10
Validation Loss: 1.2102090120315552
Validation Accuracy: 0.5137156844139099
Epoch 4/10
Validation Loss: 1.2298628091812134
Validation Accuracy: 0.4944780766963959
Epoch 5/10
Validation Loss: 1.2094892263412476
Validation Accuracy: 0.5251157879829407
Epoch 6/10
Validation Loss: 1.1563118696212769
Validation Accuracy: 0.5404345989227295
Epoch 7/10
Validation Loss: 1.1553707122802734
Validation Accuracy: 0.5571784973144531
Epoch 8/10
Validation Loss: 1.1697050333023071
Validation Accuracy: 0.5575347542762756
Epoch 9/10
Validation Loss: 1.2701401710510254
Validation Accuracy: 0.5536159873008728
Epoch 10/10
Validation Loss: 1.3532912731170654
Validation Accuracy: 0.530815839767456


In [5]:
# Evaluate on test data
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.4560 - loss: 1.7046
Test Loss: 1.369990348815918
Test Accuracy: 0.5246666669845581


In [6]:
# Load unseen images
unseen_images = []
for image_name in os.listdir(pred_path):
    image_path = os.path.join(pred_path, image_name)
    img = Image.open(image_path).resize((64, 64))
    img = np.array(img) / 255.0
    unseen_images.append(img)

unseen_images = np.array(unseen_images)

# Predict
predictions = cnn_model.predict(unseen_images)
predicted_labels = [categories[np.argmax(pred)] for pred in predictions]

# Display predictions
for i, pred in enumerate(predicted_labels):
    print(f"Image {i+1}: {pred}")

229/229 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step
Image 1: forest
Image 2: mountain
Image 3: street
Image 4: mountain
Image 5: mountain
Image 6: forest
Image 7: sea
Image 8: street
Image 9: sea
Image 10: street
Image 11: sea
Image 12: forest
Image 13: sea
Image 14: buildings
Image 15: buildings
Image 16: glacier
Image 17: forest
Image 18: forest
Image 19: mountain
Image 20: street
Image 21: forest
Image 22: forest
Image 23: street
Image 24: sea
Image 25: buildings
Image 26: buildings
Image 27: forest
Image 28: forest
Image 29: glacier
Image 30: mountain
Image 31: mountain
Image 32: buildings
Image 33: mountain
Image 34: mountain
Image 35: mountain
Image 36: sea
Image 37: sea
Image 38: glacier
Image 39: mountain
Image 40: mountain
Image 41: sea
Image 42: street
Image 43: mountain
Image 44: sea
Image 45: sea
Image 46: forest
Image 47: mountain
Image 48: sea
Image 49: street
Image 50: sea
Image 51: buildings
Image 52: mountain
Image 53: forest
Image 54: sea
Image 55: forest
Image 56: sea
Image 57